In [1]:
import pandas as pd
import numpy as np
import random
import math
from pandas import DataFrame as df
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier

----------------------------------------------------------------------------------------------------------

Início da geração do dataset. Se for importar um dataset, apenas ignore essa parte.

----------------------------------------------------------------------------------------------------------

In [2]:
QTDE_APROVADO = 500
QTDE_REPROVADO = 500

# gerar horas de estudo
horas_aprovado = np.random.normal(4, 1, QTDE_APROVADO) # centro em 4
horas_reprovado = np.random.normal(1.5, 2, QTDE_REPROVADO) # centro em 1.5
horas_reprovado = list(map(lambda x: x if x > 0 else 0, horas_reprovado)) # não pode ser menor que 0

# gerar nota de 0 a 10 dependendo das horas de estudo de forma logarítmica + ruido aleatório (alguns vão estudar muito e reprovar, outros vão estudar pouco e passar)
nota = lambda x: 8*math.log(2.5*x+1, 10) + random.uniform(-3, +3)

In [3]:
def conceito(nota):
    if nota >= 10:
        return 10.0    
    elif nota <= 0:
        return 0.0
    else:
        return nota

In [4]:
x = {}

# gerar notas
for disciplina in ('AED1','PI','IA','MD','AL'):
    notas_reprovado = list(map(lambda x: conceito(nota(x)), horas_reprovado))
    notas_aprovado = list(map(lambda x: conceito(nota(x)), horas_aprovado))
    x[disciplina] = notas_aprovado + notas_reprovado

In [5]:
# gerar matérias trancadas
trancadas_aprovado = list(np.random.randint(0,4, QTDE_APROVADO))
trancadas_reprovado = list(np.random.randint(2,6, QTDE_REPROVADO))
x['Disciplinas Trancadas'] = trancadas_aprovado + trancadas_reprovado

In [6]:
# gerar conceitos O
O_aprovado = list(np.random.randint(0,2, QTDE_APROVADO))
O_reprovado = list(np.random.randint(0,4, QTDE_REPROVADO))
x['Total de Os'] = O_aprovado + O_reprovado

In [7]:
# gerar faltas
faltas_aprovado = list(np.random.randint(0,4, QTDE_APROVADO))
faltas_reprovado = list(np.random.randint(0,6, QTDE_REPROVADO))
x['Faltas'] = faltas_aprovado + faltas_reprovado

In [8]:
# gerar CR
cr_aprovado = list(np.random.normal(2.4, 1.5, QTDE_APROVADO)) # centro em 2.4
cr_reprovado = list(np.random.normal(1.6, 1.5, QTDE_REPROVADO)) # centro em 1.6
x['CR'] = cr_aprovado + cr_reprovado

for i in range(1000):
    if x['CR'][i] < 0.0:
        x['CR'][i] = 0.0
    if x['CR'][i] > 4.0:
        x['CR'][i] = 4.0

In [9]:
# gerar a classificação
x['Class'] = list([1] * QTDE_APROVADO + [0] * QTDE_REPROVADO)
x = pd.DataFrame(x)

# mistura todas as colunas
x = x.sample(frac=1).reset_index(drop=True)

# checa se alguma célula de x está null ou NaN
print(x.isnull().sum())
print(x.isna().sum())

# imprime a matriz de correlação entre os atributos
print(x.corr())
# obviamente a correlação entre as disciplinas seria alta já que foram feitas do mesmo jeito

# exportar como csv ou ler um csv
x.to_csv(r'C:\Users\Acer\Downloads\dataset.csv', index = None, header=True)

AED1                     0
PI                       0
IA                       0
MD                       0
AL                       0
Disciplinas Trancadas    0
Total de Os              0
Faltas                   0
CR                       0
Class                    0
dtype: int64
AED1                     0
PI                       0
IA                       0
MD                       0
AL                       0
Disciplinas Trancadas    0
Total de Os              0
Faltas                   0
CR                       0
Class                    0
dtype: int64
                           AED1        PI        IA        MD        AL  \
AED1                   1.000000  0.774760  0.762420  0.756693  0.753919   
PI                     0.774760  1.000000  0.757117  0.759143  0.764420   
IA                     0.762420  0.757117  1.000000  0.746619  0.750323   
MD                     0.756693  0.759143  0.746619  1.000000  0.744658   
AL                     0.753919  0.764420  0.750323  0.7446

----------------------------------------------------------------------------------------------------------

Fim da geração do dataset.

----------------------------------------------------------------------------------------------------------

In [10]:
#x = pd.read_csv("dataset.csv")

# salva a coluna Class em y e retira de x
y = x.pop('Class')

In [11]:
# imprime os parâmetros dos dez primeiros e dez últimos alunos junto da classificação

for i in range (0,10):
    print(x.iloc[i,0:9])
    if(y[i] == 0):
        print("Reprovado")
    else:
        print("Aprovado")
        
for i in range (990,1000):
    print(x.iloc[i,0:9])
    if(y[i] == 0):
        print("Reprovado")
    else:
        print("Aprovado")
        
# apenas para ver alguns exemplos

AED1                     0.000000
PI                       0.000000
IA                       0.211219
MD                       0.000000
AL                       0.064585
Disciplinas Trancadas    5.000000
Total de Os              3.000000
Faltas                   5.000000
CR                       2.370049
Name: 0, dtype: float64
Reprovado
AED1                     0.000000
PI                       0.000000
IA                       0.000000
MD                       0.392689
AL                       0.000000
Disciplinas Trancadas    3.000000
Total de Os              1.000000
Faltas                   2.000000
CR                       2.129202
Name: 1, dtype: float64
Reprovado
AED1                     10.000000
PI                        9.876690
IA                        7.178138
MD                       10.000000
AL                       10.000000
Disciplinas Trancadas     3.000000
Total de Os               0.000000
Faltas                    0.000000
CR                        1.438129
Name:

In [12]:
# dividir em 70% treino e 30% teste (usamos cross validation no lugar)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

In [13]:
max_score = 0
best_depth = 0
best_min_sample_leaf = 0

for i in range (1,6):
    for j in range (1,30):
        #print("Max Depth = ", i,"\nMin Samples Leaf =", j)
        dt = DecisionTreeClassifier(criterion = 'gini', max_depth = i, min_samples_leaf = j, random_state = 0)
        dt_score = (cross_val_score(dt, x, y, cv=10))
        #print("Decision Tree score:", np.mean(dt_score))
        if(np.mean(dt_score) > max_score):
            max_score = np.mean(dt_score) # melhor desempenho do modelo com diferentes parâmetros para escolher os melhores
            best_depth = i; # melhor profundidade da árvore
            best_min_sample_leaf = j; # melhor quantidade mínima de amostras em nós do tipo folha
            best_dt = dt # melhor árvore

In [14]:
print("Decision tree score: ",max_score,"with depth: ",best_depth,"and min sample leaf: ",best_min_sample_leaf)

Decision tree score:  0.9460000000000001 with depth:  4 and min sample leaf:  8


In [15]:
# gera um arquivo tree.dot que pode ser aberto com o Graphviz e convertido em imagem

from sklearn.tree import export_graphviz
atributos = ["AED1", "PI", "IA", "MD", "AL", "Disciplinas Trancadas", "Total de Os", "Faltas", "CR"]
best_dt.fit(x,y)

export_graphviz(best_dt, out_file='tree.dot', 
                feature_names = atributos,
                class_names = ["Reprovado", "Aprovado"],
                rounded = True, 
                proportion = False, 
                precision = 2, 
                filled = True)